<a href="https://colab.research.google.com/github/sgr1118/GD-NLP/blob/main/%5BGD_16%5D_Hugging_Face_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 16-6. 프로젝트 : 커스텀 프로젝트 직접 만들기

- 실습 코드에서 수행해 본 내용을 토대로, 이번에는 GLUE dataset의 mnli task를 수행하는 프로젝트를 커스텀 프로젝트 형태로 진행해 봅시다.

- mnli task는 이전 스텝에서 사용한 BERT를 사용하면 학습이 제대로 되지 않습니다. [여기](https://huggingface.co/models)를 참조하여 BERT가 아닌 다른 모델을 선택하세요. tensorflow와 해당 모델에 대한 task로 검색하면 사용할 수 있는 모델이 나옵니다. 그 후 선택한 모델의 _tokenizer_와 해당 모델에 대한 task 와 모델 의 정보를 [여기](https://huggingface.co/docs/transformers/index)에서 찾아 여러분의 프로젝트를 완성해 보세요.

- 그냥 run_glue.py를 돌려보는 방식으로 진행하는 것을 원하는 것은 아닙니다. 아래와 같은 순서를 지켜서 진행해 주세요

1. 라이브러리 버전을 확인해 봅니다.

In [ ]:
import transformers
import os
import warnings
import numpy as np
import argparse
from argparse import ArgumentParser
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(tf.__version__)
print(np.__version__)
print(transformers.__version__)
print(argparse.__version__)

2.6.0
1.21.4
4.11.3
1.1


In [ ]:
DEPRECATION_WARNING = (
    "This {0} will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets "
    "library. You can have a look at this example script for pointers: "
    "https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py")

# STEP 1. mnli 데이터셋을 분석해 보기

- tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 올려야 합니다.

In [ ]:
!pip install tensorflow-datasets -U

     |████████████████████████████████| 4.7 MB 7.2 MB/s            
     |████████████████████████████████| 127 kB 83.8 MB/s            
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.4.0
    Uninstalling tensorflow-datasets-4.4.0:
      Successfully uninstalled tensorflow-datasets-4.4.0


In [ ]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mnli/2.0.0
INFO:absl:Load dataset info from /tmp/tmp4thpf0xxtfds
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

INFO:absl:Downloading https://dl.fbaipublicfiles.com/glue/data/MNLI.zip into /aiffel/tensorflow_datasets/downloads/dl.fbaipublicfiles.com_glue_MNLIdNe8cK2kTBCG0bqBz2JxwShRT2KfuO3NVIwROTnjtfI.zip.tmp.878a1dbca1584f0e8a6934cfc4813665...


...

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

INFO:absl:Done writing glue-test_mismatched.tfrecord. Number of examples: 9847 (shards: [9847])
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

In [ ]:
import datasets
from datasets import load_dataset

huggingface_mnli_dataset = load_dataset('glue', 'mnli')
print(huggingface_mnli_dataset)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [ ]:
huggingface_mnli_dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")

model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

huggingface_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [ ]:
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [ ]:
data

{Split('train'): <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>}

In [ ]:
# 데이터셋 안에 어떤 항목이 정의되어 있는지 확인

examples = data['train'].take(1)
for example in examples:
    premise = example['premise']
    hypothesis = example['hypothesis']
    label = example['label']
    print(premise)
    print(hypothesis)
    print(label)

tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [ ]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

# STEP 2. MNLIProcessor클래스 구현하기

In [ ]:
class MNLIProcessor(DataProcessor):
    """Processor for the MNLI data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            tensor_dict["premise"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[8]
            text_b = line[9]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [ ]:
# 원본 데이터셋을 처리하여 모델에 입력할 수 있도록 정리해 주는 클래스

class MrpcProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            tensor_dict["premise"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[8]
            text_b = line[9]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [ ]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [ ]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='entailment')


In [ ]:
# 실제 label을 확인하여 Binary Classification 문제로 잘 정의되고 있는지 확인
label_list = processor.get_labels()
label_list

['contradiction', 'entailment', 'neutral']

In [ ]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

# STEP 3. 위에서 구현한 processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
huggingface_tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")

huggingface_model = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

In [ ]:
def transform(data):
  return huggingface_tokenizer(
      data['premise'],
      data['hypothesis'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      )
  
examples = huggingface_mnli_dataset['train'][:5]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'One of our number will carry out your instructions minutely.', 'How do you know? All this is their information again.', "yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range"], 'hypothesis': ['Product and geography are what make cream skimming work. ', 'You lose the things to the following level if the people recall.', 'A member of my team will execute your orders with immense precision.', 'This information belongs to them.', 'The tennis shoes have a range of prices.'], 'label': [1, 0, 0, 0, 1], 'idx': [0, 1, 2, 3, 4]}
{'inp

In [ ]:
encoded_dataset = huggingface_mnli_dataset.map(transform, batched=True)

  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [ ]:
def tf_glue_convert_examples_to_features(examples, 
                                         tokenizer, 
                                         max_length, 
                                         processor, 
                                         label_list=None, 
                                         output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

In [ ]:
# 최종적으로 모델에 전달될 tf.data.Dataset 인스턴스를 생성
train_dataset = tf_glue_convert_examples_to_features(data['train'], 
                                                     tokenizer, max_length=128, processor=processor)

In [ ]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1999,  5038,  1997,  2122, 13136,  1010,  1048, 11020,
        2038,  2499, 29454, 29206, 14626,  2144,  2786,  2000, 16636,
        1996, 10908,  1997,  1996,  2110,  4041,  6349,  1998,  2000,
        5323, 15902, 13797,  2007, 22859,  6461,  2012,  6469,  2075,
        1037,  2047, 25353, 14905, 10735,  2483,  2090,  1996,  2976,
       10802,  1998, 15991,  1997,  3423,  2578,  4804,  1012,   102,
       15902, 13797,  2007, 22859,  2003, 10232,  1012,   102,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

In [ ]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

In [ ]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

In [ ]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

# STEP 4. model을 생성하여 학습 및 테스트를 진행해 보기

- 혹시 STEP 2의 진행에 어려움을 겪고 계신다면 transformer 프로젝트 내부를 살펴보시면 참고할만한 예시 코드를 찾아볼 수 있을 것입니다. transformers의 공식 [github](https://github.com/huggingface/transformers)을 참고하는 것도 좋은 방법이에요!

In [ ]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 이미 잘 훈련된 BERT 모델을 가져다가 fine-tuning하는 작업

# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset, 
          epochs=3, 
          steps_per_epoch=115, 
          validation_data=validation_dataset_batch)

Epoch 1/3


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.9/site-pack

In [ ]:
# Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = os.getenv('HOME')+'/aiffel/transformers'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 8, # 각 device 당 batch size
    per_device_eval_batch_size = 8, # evaluation 시에 batch size
    num_train_epochs = 1, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
)

In [ ]:
from datasets import load_metric
metric = load_metric('glue', 'mnli')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=huggingface_model,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_dataset['train'],    # training dataset
    eval_dataset=encoded_dataset['validation_matched'],       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise.
***** Running training *****
  Num examples = 392702
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 49088


Epoch,Training Loss,Validation Loss,Accuracy
1,0.292800,0.930875,0.813653


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-18000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-18000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-18000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-18500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-18500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-18500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-19000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-19000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-19000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-19500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-19500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-19500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transf

Model weights saved in /aiffel/aiffel/transformers/checkpoint-35000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-35500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-35500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-35500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-36000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-36000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-36000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-36500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-36500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-36500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-37000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-37000/config.json
Model weights saved in /aiffel/aiffel/transforme

TrainOutput(global_step=49088, training_loss=0.29948605905922016, metrics={'train_runtime': 21341.9578, 'train_samples_per_second': 18.4, 'train_steps_per_second': 2.3, 'total_flos': 5.202114009364685e+16, 'train_loss': 0.29948605905922016, 'epoch': 1.0})

In [ ]:
trainer.evaluate(encoded_dataset['test_matched'])

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise.
***** Running Evaluation *****
  Num examples = 9796
  Batch size = 8
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCri

RuntimeError: CUDA error: device-side assert triggered

# 회고

1. 이번 프로젝트에서 어려웠던 점
 
(1) CUDA 메모리 오류가 계속 떠 학습 과정이 쉽지않았습니다.

2. 프로젝트를 진행하면서 알게된 부분 또는 아직 이해하지 못한 부분

(1) CUDA 메모리 오류를 해결하기 위한 방법
- batch_size 조절
- 더 작은 모델 불러오기






